# IS4242 Group Assignment Part 1
**November 11, 2020**

## Changelog from submission 3 to submission 4
- Filtered additional predictors to reduce redundancy
- Reduced grouping of non-missing data to others as it might have introduced more discrepancies
- Added AX to be used with RF
#### Explanations at their relevant code portions

#### Name: LECK WEI SHENG IAN
#### NUS ID: A0168177R
#### Name: WOO KENG THONG
#### NUS ID: A0167991L

Your goal is to predict the operating condition of a waterpoint for each record in the dataset. You are provided information about the waterpoints in order label them.

The labels in this dataset are simple. There are three possible values:
1. functional - the waterpoint is operational and there are no repairs needed
2. functional needs repair - the waterpoint is operational, but needs repairs
3. non functional - the waterpoint is not operational

The format for the submission file is simply the row id and the predicted label. 
- id	status_group
- 50785	functional
- 51630	functional

CSV would thus look like
- id,status_group
- 50785,functional
- 51630,functional

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Preprocessing & Feature Engineering

Load data, and merge data and labels together into one dataframe

In [2]:
labels = pd.read_csv('training-set-labels.csv')
df = pd.read_csv('training-set-values.csv')
test_df = pd.read_csv('test-set-values.csv')

df = pd.merge(df, labels, on='id')

Explore data set

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Check similar predictors and eliminate highly correlated predictors to reduce redundancy. <br>
Given that random forest will be used, highly correlated features might mask interactions between features and cause the accuracy to suffer. Hence, more features will be examined for correlation and subsequently removed to improve the random forest performance for the fourth submission.
https://datascience.stackexchange.com/questions/24452/in-supervised-learning-why-is-it-bad-to-have-correlated-features

In [4]:
df.groupby(['region','region_code']).size()

region         region_code
Arusha         2              3024
               24              326
Dar es Salaam  7               805
Dodoma         1              2201
Iringa         11             5294
Kagera         18             3316
Kigoma         16             2816
Kilimanjaro    3              4379
Lindi          8               300
               18                8
               80             1238
Manyara        21             1583
Mara           20             1969
Mbeya          12             4639
Morogoro       5              4006
Mtwara         9               390
               90              917
               99              423
Mwanza         17               55
               19             3047
Pwani          6              1609
               40                1
               60             1025
Rukwa          15             1808
Ruvuma         10             2640
Shinyanga      11                6
               14               20
               17           

Drop `region_code` as it seems to be identify regions, yet is not able to stand on its own as there are identical region codes in different regions. `drop` list is compiled for each column dropped for subsequent use with test values.

In [5]:
drop = []
drop.append('region_code')

df.drop('region_code',axis=1,inplace=True)
drop

['region_code']

In [6]:
df.groupby(['extraction_type','extraction_type_group','extraction_type_class']).size()

extraction_type            extraction_type_group  extraction_type_class
afridev                    afridev                handpump                  1770
cemo                       other motorpump        motorpump                   90
climax                     other motorpump        motorpump                   32
gravity                    gravity                gravity                  26780
india mark ii              india mark ii          handpump                  2400
india mark iii             india mark iii         handpump                    98
ksb                        submersible            submersible               1415
mono                       mono                   motorpump                 2865
nira/tanira                nira/tanira            handpump                  8154
other                      other                  other                     6430
other - mkulima/shinyanga  other handpump         handpump                     2
other - play pump          other hand

Drop `extraction_type_group` as it seems to be a repeat of either of `extraction_type` or `extraction_type_class`. It seems to only identify differently when it is `other ...`, where `...` is `handpump` or `motorpump` for example, based on `extraction_type_class` value.

In [7]:
drop.append('extraction_type_group')

df.drop('extraction_type_group',axis=1,inplace=True)
drop

['region_code', 'extraction_type_group']

In [8]:
df.groupby(['management','management_group']).size()

management        management_group
company           commercial            685
other             other                 844
other - school    other                  99
parastatal        parastatal           1768
private operator  commercial           1971
trust             commercial             78
unknown           unknown               561
vwc               user-group          40507
water authority   commercial            904
water board       user-group           2933
wua               user-group           2535
wug               user-group           6515
dtype: int64

`management` appears to provide more specific information than `management_group`, which only appears to show the category of `management` - `commercial` for example. Hence `management_group` will be dropped.

In [9]:
drop.append('management_group')

df.drop('management_group',axis=1,inplace=True)
drop

['region_code', 'extraction_type_group', 'management_group']

In [10]:
df.groupby(['payment','payment_type']).size()

payment                payment_type
never pay              never pay       25348
other                  other            1054
pay annually           annually         3642
pay monthly            monthly          8300
pay per bucket         per bucket       8985
pay when scheme fails  on failure       3914
unknown                unknown          8157
dtype: int64

`payment` is highly correlated to `payment_type` $-$ which appears to categorize `payment` $-$ but does not offer more specific information. Hence it will thus be dropped.

In [11]:
drop.append('payment')

df.drop('payment',axis=1,inplace=True)
drop

['region_code', 'extraction_type_group', 'management_group', 'payment']

In [12]:
df.groupby(['water_quality','quality_group']).size()

water_quality       quality_group
coloured            colored            490
fluoride            fluoride           200
fluoride abandoned  fluoride            17
milky               milky              804
salty               salty             4856
salty abandoned     salty              339
soft                good             50818
unknown             unknown           1876
dtype: int64

`quality_group` is highly correlated to `water_quality`, but is less specific and provides less information. Hence, it will be dropped.

In [13]:
drop.append('quality_group')

df.drop('quality_group', axis=1, inplace=True)
drop

['region_code',
 'extraction_type_group',
 'management_group',
 'payment',
 'quality_group']

In [14]:
df.groupby(['quantity','quantity_group']).size()

quantity      quantity_group
dry           dry                6246
enough        enough            33186
insufficient  insufficient      15129
seasonal      seasonal           4050
unknown       unknown             789
dtype: int64

`quantity` is highly correlated to `quantity_group` $-$ which appears to categorize `quantity` $-$ but does not offer more specific information and will thus be dropped.

In [15]:
drop.append('quantity')

df.drop('quantity',axis=1,inplace=True)
drop

['region_code',
 'extraction_type_group',
 'management_group',
 'payment',
 'quality_group',
 'quantity']

In [16]:
df.groupby(['source','source_type','source_class']).size()

source                source_type           source_class
dam                   dam                   surface           656
hand dtw              borehole              groundwater       874
lake                  river/lake            surface           765
machine dbh           borehole              groundwater     11075
other                 other                 unknown           212
rainwater harvesting  rainwater harvesting  surface          2295
river                 river/lake            surface          9612
shallow well          shallow well          groundwater     16824
spring                spring                groundwater     17021
unknown               other                 unknown            66
dtype: int64

`source_type` appears to be highly correlated with `source`, but is less specific. `source_class` is a further grouping of `source` into three types $-$ `surface`, `groundwater` and `unknown`. Hence, both `source_type` and `source_class` will be dropped.

In [17]:
drop.append('source_type')
drop.append('source_class')

df.drop('source_type',axis=1,inplace=True)
df.drop('source_class',axis=1,inplace=True)
drop

['region_code',
 'extraction_type_group',
 'management_group',
 'payment',
 'quality_group',
 'quantity',
 'source_type',
 'source_class']

In [18]:
df.groupby(['waterpoint_type','waterpoint_type_group']).size()

waterpoint_type              waterpoint_type_group
cattle trough                cattle trough              116
communal standpipe           communal standpipe       28522
communal standpipe multiple  communal standpipe        6103
dam                          dam                          7
hand pump                    hand pump                17488
improved spring              improved spring            784
other                        other                     6380
dtype: int64

`waterpoint_type_group` is highly correlated to `waterpoint_type`, but is less specific and provides less information. Hence, it will be dropped.

In [19]:
drop.append('waterpoint_type_group')

df.drop('waterpoint_type_group',axis=1,inplace=True)
drop

['region_code',
 'extraction_type_group',
 'management_group',
 'payment',
 'quality_group',
 'quantity',
 'source_type',
 'source_class',
 'waterpoint_type_group']

In [20]:
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,80.000000,30500.000000,2013.000000


Remove `num_private` as it does not seem to be meaningful - mostly zeros at 25%, 50% and 75%.

In [21]:
drop.append('num_private')

df.drop('num_private',axis=1,inplace=True)
drop

['region_code',
 'extraction_type_group',
 'management_group',
 'payment',
 'quality_group',
 'quantity',
 'source_type',
 'source_class',
 'waterpoint_type_group',
 'num_private']

Check for null values in data

In [22]:
df.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
basin                        0
subvillage                 371
region                       0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_class        0
management                   0
payment_type                 0
water_quality                0
quantity_group               0
source                       0
waterpoint_type              0
status_group                 0
dtype: int64

Deal with columns containing null values.
- #### funder

In [23]:
df['funder'].value_counts().head(20)

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
Kkkt                      1287
World Vision              1246
Unicef                    1057
Tasaf                      877
District Council           843
Dhv                        829
Private Individual         826
Dwsp                       811
0                          777
Norad                      765
Germany Republi            610
Tcrs                       602
Ministry Of Water          590
Water                      583
Dwe                        484
Name: funder, dtype: int64

~Keep top 5 `funder` and set the rest to `other`, including missing values.<br>
Using only the main 5 `funder` and reverting the rest to `other` is to reduce computational cost and mitigate overfitting.~ <br>
We realised that the top 5 may not sufficiently represent all the different types of funders, and hence will only be replacing missing data with `other`.

In [24]:
df['funder'].fillna('other', inplace=True)

Deal with columns containing null values.
- #### installer

In [25]:
df.installer.value_counts().head(20)

DWE                   17402
Government             1825
RWE                    1206
Commu                  1060
DANIDA                 1050
KKKT                    898
Hesawa                  840
0                       777
TCRS                    707
Central government      622
CES                     610
Community               553
DANID                   552
District Council        551
HESAWA                  539
LGA                     408
World vision            408
WEDECO                  397
TASAF                   396
District council        392
Name: installer, dtype: int64

~Keep top 5 `installer` and set the rest to `other`, including missing values.<br>
Using only the main 5 `installer` and reverting the rest to `other` is to try to reduce computational cost and mitigate overfitting.~<br>
We realised that the top 5 may not sufficiently represent all the different types of funders, and hence will only be replacing missing data with `other`.

In [26]:
df['installer'].fillna('other',inplace=True)

Deal with columns containing null values.
- #### subvillage

In [27]:
df.subvillage.value_counts()

Madukani            508
Shuleni             506
Majengo             502
Kati                373
Mtakuja             262
                   ... 
Lugese A              1
Mungoma Ya Chini      1
Kalemela B Kati       1
Murutunguru           1
Barabara Kuu A        1
Name: subvillage, Length: 19287, dtype: int64

There are 19287 unique `subvillage`, of which the largest group is only 508. As the total dataset only has around 59400 values, about a third of the data is unique. It is thus unlikely to be a meaningful feature, and will be dropped.

In [28]:
drop.append('subvillage')
df.drop('subvillage',axis=1,inplace=True)
drop

['region_code',
 'extraction_type_group',
 'management_group',
 'payment',
 'quality_group',
 'quantity',
 'source_type',
 'source_class',
 'waterpoint_type_group',
 'num_private',
 'subvillage']

Deal with columns containing null values.
- #### public_meeting

In [29]:
df.public_meeting.value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

In [30]:
df.groupby(['public_meeting','status_group']).size()

public_meeting  status_group           
False           functional                  2173
                functional needs repair      442
                non functional              2440
True            functional                 28408
                functional needs repair     3719
                non functional             18884
dtype: int64

Convert `public_meeting` to binary predictor and impute with mode.

In [31]:
def convert_public_meeting(row):
    if row['public_meeting']==True:
        return 1
    elif row['public_meeting']==False:
        return 0
    else:
        return np.nan
    
df['public_meeting'] = df.apply(lambda row: convert_public_meeting(row), axis=1)
df['public_meeting'].fillna(df['public_meeting'].mode().item(),inplace=True)
df.groupby(['public_meeting','status_group']).size()

public_meeting  status_group           
0.0             functional                  2173
                functional needs repair      442
                non functional              2440
1.0             functional                 30086
                functional needs repair     3875
                non functional             20384
dtype: int64

Deal with columns containing null values.
- #### scheme_management

In [32]:
df.scheme_management.value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

~Keep top 5 `scheme_management` and set the rest to `other`, including missing values.<br>
Using only the main 5 `scheme_management` and reverting the rest to `other` is to try to reduce computational cost and mitigate overfitting.~<br>
We realised that there is no need to reduce to 5 features, as there are only a total of 12 types of `scheme_management`. Hence, we will only be replacing missing data with `other`.

In [33]:
df['scheme_management'].fillna('other',inplace=True)

Deal with columns containing null values.
- #### scheme_name

In [34]:
df.scheme_name.value_counts()

K                              682
None                           644
Borehole                       546
Chalinze wate                  405
M                              400
                              ... 
Namba 7 dam                      1
Kaseni water supply              1
Ikovo                            1
Tungu windmill piped scheme      1
Burieni water supply             1
Name: scheme_name, Length: 2696, dtype: int64

There are 2696 unique `scheme_name`, of which the largest group is only 682. Additionally, there are 28166 null values in this column. As the total dataset only has around 59400 values, nearly half of the data is either unique or missing. It is thus unlikely to be a meaningful feature, and will be dropped.

In [35]:
drop.append('scheme_name')

df.drop('scheme_name',axis=1,inplace=True)
drop

['region_code',
 'extraction_type_group',
 'management_group',
 'payment',
 'quality_group',
 'quantity',
 'source_type',
 'source_class',
 'waterpoint_type_group',
 'num_private',
 'subvillage',
 'scheme_name']

Deal with columns containing null values.
- #### permit

In [36]:
df.permit.value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [37]:
df.groupby(['permit','status_group']).size()

permit  status_group           
False   functional                  9045
        functional needs repair     1320
        non functional              7127
True    functional                 21541
        functional needs repair     2697
        non functional             14614
dtype: int64

Convert `permit` to binary predictor and impute with mode.

In [38]:
def convert_permit(row):
    if row['permit']==True:
        return 1
    elif row['permit']==False:
        return 0
    else:
        return np.nan
    
df['permit'] = df.apply(lambda row: convert_permit(row), axis=1)
df['permit'].fillna(df['permit'].mode().item(),inplace=True)
df.groupby(['permit','status_group']).size()

permit  status_group           
0.0     functional                  9045
        functional needs repair     1320
        non functional              7127
1.0     functional                 23214
        functional needs repair     2997
        non functional             15697
dtype: int64

Having removed all null, ensure that there is no other invalid data $-$ 0 $-$ that can be immediately obvious for relevant columns such as `population`, `gps_height`, `amount_tsh` and `construction_year`.

In [39]:
df['gps_height'].replace(0, np.nan, inplace=True)
df['population'].replace(0, np.nan, inplace=True)
df['amount_tsh'].replace(0, np.nan, inplace=True)
df['construction_year'].replace(0, np.nan, inplace=True)
df.isnull().sum()

id                           0
amount_tsh               41639
date_recorded                0
funder                       0
gps_height               20438
installer                    0
longitude                    0
latitude                     0
wpt_name                     0
basin                        0
region                       0
district_code                0
lga                          0
ward                         0
population               21381
public_meeting               0
recorded_by                  0
scheme_management            0
permit                       0
construction_year        20709
extraction_type              0
extraction_type_class        0
management                   0
payment_type                 0
water_quality                0
quantity_group               0
source                       0
waterpoint_type              0
status_group                 0
dtype: int64

`gps_height` is likely to be location-dependent as it is the height of an area is likely to be similar.<br>
`amount_tsh` is also likely to be location-dependent as the amount of water that can be drawn in the same area is likely to be similar.<br>
`population` is also likely to be location-dependent as communities in the same area will be subject to similar living conditions.<br>
Holding the above assumptions, we will assume that the three predictors are affected by `region` and `district_code`, which based on the name, indicate a general area.<br>
Hence, `region` and `district_code` will also be used when imputing missing data with mean values for `gps_height`, `amount_tsh` and `population`.<br>

#### There is no clear indication whether the other predictors imputed earlier are relevant to geographic area, and hence were not imputed with `region`/`district_code`.

In [40]:
df['amount_tsh'].fillna(df.groupby(['region', 'district_code'])['amount_tsh'].transform('mean'), inplace=True)
df['amount_tsh'].fillna(df.groupby(['region'])['amount_tsh'].transform('mean'), inplace=True)
df['amount_tsh'].fillna(df['amount_tsh'].mean(), inplace=True)
df['gps_height'].fillna(df.groupby(['region', 'district_code'])['gps_height'].transform('mean'), inplace=True)
df['gps_height'].fillna(df.groupby(['region'])['gps_height'].transform('mean'), inplace=True)
df['gps_height'].fillna(df['gps_height'].mean(), inplace=True)
df['population'].fillna(df.groupby(['region', 'district_code'])['population'].transform('mean'), inplace=True)
df['population'].fillna(df.groupby(['region'])['population'].transform('mean'), inplace=True)
df['population'].fillna(df['population'].mean(), inplace=True)

`construction_year` (as a numeric predictor) is also imputed with mean value. We assume that there is no relation between region and construction year as it is unlikely construction within each geographic area occurs in the same period due to resource constraint.

In [41]:
df['construction_year'].fillna(df['construction_year'].mean(),inplace=True)

df.isnull().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
basin                    0
region                   0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_class    0
management               0
payment_type             0
water_quality            0
quantity_group           0
source                   0
waterpoint_type          0
status_group             0
dtype: int64

`construction_year` provides the year that it is constructed and it is a good source for feature engineering. The longer a water point is operational, the more likely it is for the water point to be non functional or needs repair. <br> Convert `construction_year` and `date_recorded` into the number of years the waterpoint has been in operation for, and drop both features after as the `operational years` is likely to be a more useful predictor

In [42]:
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
df['operational_years'] = df.date_recorded.dt.year - df.construction_year

df.drop('date_recorded', axis=1, inplace=True)
df.drop('construction_year', axis=1, inplace=True)
drop.append('date_recorded')
drop.append('construction_year')

## Take the same pre-processing steps for test data

In [43]:
test_df.isnull().sum()

id                          0
amount_tsh                  0
date_recorded               0
funder                    869
gps_height                  0
installer                 877
longitude                   0
latitude                    0
wpt_name                    0
num_private                 0
basin                       0
subvillage                 99
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting            821
recorded_by                 0
scheme_management         969
scheme_name              7092
permit                    737
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity  

Fill missing data in test dataset

In [44]:
df['funder'].fillna('other', inplace=True)
df['installer'].fillna('other', inplace=True)
df['scheme_management'].fillna('other', inplace=True)

In [45]:
test_df['public_meeting'] = test_df.apply(lambda row: convert_public_meeting(row), axis=1)
test_df['public_meeting'].fillna(test_df['public_meeting'].mode().item(),inplace=True)

test_df['permit'] = test_df.apply(lambda row: convert_permit(row), axis=1)
test_df['permit'].fillna(test_df['permit'].mode().item(),inplace=True)

Having removed all null, ensure that there is no other invalid data $-$ 0 $-$ that can be immediately obvious for relevant columns such as `population`, `gps_height`, `amount_tsh` and `construction_year`.

In [46]:
test_df['gps_height'].replace(0, np.nan, inplace=True)
test_df['population'].replace(0, np.nan, inplace=True)
test_df['amount_tsh'].replace(0, np.nan, inplace=True)
test_df['construction_year'].replace(0, np.nan, inplace=True)
test_df.isnull().sum()

id                           0
amount_tsh               10410
date_recorded                0
funder                     869
gps_height                5211
installer                  877
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                  99
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                5453
public_meeting               0
recorded_by                  0
scheme_management          969
scheme_name               7092
permit                       0
construction_year         5260
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

`gps_height` is likely to be location-dependent as it is the height of an area is likely to be similar.<br>
`amount_tsh` is also likely to be location-dependent as the amount of water that can be drawn in the same area is likely to be similar.<br>
`population` is also likely to be location-dependent as communities in the same area will be subject to similar living conditions.<br>
Holding the above assumptions, we will assume that the three predictors are affected by `region` and `district_code`, which based on the name, indicate a general area.<br>
Hence, `region` and `district_code` will also be used when imputing missing data with mean values for `gps_height`, `amount_tsh` and `population`.<br>

#### There is no clear indication whether the other predictors imputed earlier are relevant to geographic area, and hence were not imputed with `region`/`district_code`.

In [47]:
test_df['amount_tsh'].fillna(test_df.groupby(['region', 'district_code'])['amount_tsh'].transform('mean'), inplace=True)
test_df['amount_tsh'].fillna(test_df.groupby(['region'])['amount_tsh'].transform('mean'), inplace=True)
test_df['amount_tsh'].fillna(test_df['amount_tsh'].mean(), inplace=True)
test_df['gps_height'].fillna(test_df.groupby(['region', 'district_code'])['gps_height'].transform('mean'), inplace=True)
test_df['gps_height'].fillna(test_df.groupby(['region'])['gps_height'].transform('mean'), inplace=True)
test_df['gps_height'].fillna(test_df['gps_height'].mean(), inplace=True)
test_df['population'].fillna(test_df.groupby(['region', 'district_code'])['population'].transform('mean'), inplace=True)
test_df['population'].fillna(test_df.groupby(['region'])['population'].transform('mean'), inplace=True)
test_df['population'].fillna(test_df['population'].mean(), inplace=True)

`construction_year` (as a numeric predictor) is also imputed with mean value. We assume that there is no relation between region and construction year as it is unlikely construction within each geographic area occurs in the same period due to resource constraint.

In [48]:
test_df['construction_year'].fillna(test_df['construction_year'].mean(),inplace=True)

`construction_year` provides the year that it is constructed and it is a good source for feature engineering. The longer a water point is operational, the more likely it is for the water point to be non functional or needs repair. <br> Convert `construction_year` and `date_recorded` into the number of years the waterpoint has been in operation for, and drop both features after as the `operational years` is likely to be a more useful predictor

In [49]:
test_df['date_recorded'] = pd.to_datetime(test_df['date_recorded'])
test_df['operational_years'] = test_df.date_recorded.dt.year - test_df.construction_year

Drop columns

In [50]:
for i in drop:
    test_df.drop(i, axis=1, inplace=True)

Export preprocessed data

In [51]:
pd.DataFrame(df).to_csv("clean.csv", index=False)
pd.DataFrame(test_df).to_csv("clean_test.csv", index=False)

## Model building (Fourth Submission) - RF with AX

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

Read Data

In [53]:
X_train = pd.read_csv('clean.csv')
y_train = X_train.pop('status_group')
X_test = pd.read_csv('clean_test.csv')

In [54]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   funder                 59400 non-null  object 
 3   gps_height             59400 non-null  float64
 4   installer              59400 non-null  object 
 5   longitude              59400 non-null  float64
 6   latitude               59400 non-null  float64
 7   wpt_name               59400 non-null  object 
 8   basin                  59400 non-null  object 
 9   region                 59400 non-null  object 
 10  district_code          59400 non-null  int64  
 11  lga                    59400 non-null  object 
 12  ward                   59400 non-null  object 
 13  population             59400 non-null  float64
 14  public_meeting         59400 non-null  float64
 15  re

### Encode labels into categorical variables
https://stackoverflow.com/questions/40336502/want-to-know-the-diff-among-pd-factorize-pd-get-dummies-sklearn-preprocessing/40338956

In [55]:
X_train['funder'] = pd.factorize(X_train['funder'])[0]
X_train['installer'] = pd.factorize(X_train['installer'])[0]
X_train['wpt_name'] = pd.factorize(X_train['wpt_name'])[0]
X_train['basin'] = pd.factorize(X_train['basin'])[0]
X_train['region'] = pd.factorize(X_train['region'])[0]
X_train['lga'] = pd.factorize(X_train['lga'])[0]
X_train['ward'] = pd.factorize(X_train['ward'])[0]
X_train['recorded_by'] = pd.factorize(X_train['recorded_by'])[0]
X_train['scheme_management'] = pd.factorize(X_train['scheme_management'])[0]
X_train['extraction_type'] = pd.factorize(X_train['extraction_type'])[0]
X_train['extraction_type_class'] = pd.factorize(X_train['extraction_type_class'])[0]
X_train['management'] = pd.factorize(X_train['management'])[0]
X_train['payment_type'] = pd.factorize(X_train['payment_type'])[0]
X_train['water_quality'] = pd.factorize(X_train['water_quality'])[0]
X_train['quantity_group'] = pd.factorize(X_train['quantity_group'])[0]
X_train['source'] = pd.factorize(X_train['source'])[0]
X_train['waterpoint_type'] = pd.factorize(X_train['waterpoint_type'])[0]

In [56]:
X_test['funder'] = pd.factorize(X_test['funder'])[0]
X_test['installer'] = pd.factorize(X_test['installer'])[0]
X_test['wpt_name'] = pd.factorize(X_test['wpt_name'])[0]
X_test['basin'] = pd.factorize(X_test['basin'])[0]
X_test['region'] = pd.factorize(X_test['region'])[0]
X_test['lga'] = pd.factorize(X_test['lga'])[0]
X_test['ward'] = pd.factorize(X_test['ward'])[0]
X_test['recorded_by'] = pd.factorize(X_test['recorded_by'])[0]
X_test['scheme_management'] = pd.factorize(X_test['scheme_management'])[0]
X_test['extraction_type'] = pd.factorize(X_test['extraction_type'])[0]
X_test['extraction_type_class'] = pd.factorize(X_test['extraction_type_class'])[0]
X_test['management'] = pd.factorize(X_test['management'])[0]
X_test['payment_type'] = pd.factorize(X_test['payment_type'])[0]
X_test['water_quality'] = pd.factorize(X_test['water_quality'])[0]
X_test['quantity_group'] = pd.factorize(X_test['quantity_group'])[0]
X_test['source'] = pd.factorize(X_test['source'])[0]
X_test['waterpoint_type'] = pd.factorize(X_test['waterpoint_type'])[0]

Tune hyperparameters

Adding more parameters to be tuned automatically in comparison to just `n_estimators` in the previous submission.<br>

We are using `oob_score_` to optimize as it is generally unbiased, whereas $R^2$ might overfit on training data.
https://stats.stackexchange.com/questions/288699/r2-score-vs-oob-score-random-forest

In [57]:
def rf_ax_score(parameterization, weight=None):

    p_names = ['max_features', 'min_samples_split',
              'min_samples_leaf', 'n_estimators']
    params = {}
    
    for p in p_names:
        params[p] = parameterization.get(p)
    
    print(params)

    rf = RandomForestClassifier(criterion='gini',
                                max_features=params['max_features'],
                                min_samples_split=params['min_samples_split'],
                                min_samples_leaf=params['min_samples_leaf'],
                                n_estimators=params['n_estimators'],
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)
    rf.fit(X_train, y_train)
    
    print(rf.oob_score_)
    return rf.oob_score_

def evaluate_rf(parameters):
    return {"rf_ax": rf_ax_score(parameters)}

`max_features` is no longer `auto` to allow us to find the ideal number of `max_features` during auto hyperparameter tuning.<br>
`min_samples_split` is bound between default 2 and 10 to find the optimum amount, while mitigating over-fitting by having a potentially higher `min_samples_split`.<br>
`min_samples_leaf` is bound between default 1 and 20, as a smaller leaf might make the forest more prone to capturing noise and 'growing' in a wrong direction.
https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/<br>
`n_estimators` is set between 100 and 1500, similar to the previous range. 

In [58]:
parameters=[
    {
        "name": "max_features",
        "type": "range",
        "bounds": [1, 26],
        "log_scale": False,
    },
    {
        "name": "min_samples_split",
        "type": "range",
        "bounds": [2, 10],
    },
    {
        "name": "min_samples_leaf",
        "type": "range",
        "bounds": [1, 20],
    },
    {
        "name": "n_estimators",
        "type": "range",
        "bounds": [100, 1500],
    },
]

In [59]:
from ax import optimize

best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=rf_ax_score,
    objective_name='rf_ax',
    total_trials=15
)

[INFO 11-11 11:52:46] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter max_features. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-11 11:52:46] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter min_samples_split. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-11 11:52:46] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter min_samples_leaf. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-11 11:52:46] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter n_estimators. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' 

{'max_features': 26, 'min_samples_split': 7, 'min_samples_leaf': 1, 'n_estimators': 1044}


[INFO 11-11 11:54:38] ax.service.managed_loop: Running optimization trial 2...


0.8146969696969697
{'max_features': 17, 'min_samples_split': 2, 'min_samples_leaf': 17, 'n_estimators': 737}


[INFO 11-11 11:55:24] ax.service.managed_loop: Running optimization trial 3...


...

0.8150841750841751
{'max_features': 22, 'min_samples_split': 7, 'min_samples_leaf': 1, 'n_estimators': 1500}


[INFO 11-11 12:10:36] ax.service.managed_loop: Running optimization trial 14...


0.8147811447811448
{'max_features': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'n_estimators': 1092}


[INFO 11-11 12:12:07] ax.service.managed_loop: Running optimization trial 15...


0.8156902356902357
{'max_features': 16, 'min_samples_split': 10, 'min_samples_leaf': 1, 'n_estimators': 1500}
0.8156565656565656


In [60]:
best_parameters

{'max_features': 20,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'n_estimators': 1092}

In [61]:
values

({'rf_ax': 0.815690232858758}, {'rf_ax': {'rf_ax': 1.08241320505782e-11}})

Fit random forest with best features

In [62]:
X_train.drop('id',axis=1,inplace=True)
rf = RandomForestClassifier(criterion='gini',
                                max_features=best_parameters['max_features'],
                                min_samples_split=best_parameters['min_samples_split'],
                                min_samples_leaf=best_parameters['min_samples_leaf'],
                                n_estimators=best_parameters['n_estimators'],
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)
                            
rf.fit(X_train, y_train.values.ravel())
print("%.4f" % rf.oob_score_)

0.8156


Make prediction on test data

In [63]:
idx=X_test['id']
X_test.drop(['id'],axis=1, inplace=True)
y_pred = rf.predict(X_test)

Create new dataframe with predictions and id

In [64]:
y_pred=pd.DataFrame(y_pred)
y_pred['id']=idx
y_pred.columns=['status_group','id']
y_pred=y_pred[['id','status_group']]

Create new csv for submission

In [65]:
pd.DataFrame(y_pred).to_csv("submission_rf_ax.csv", index=False)